In [1]:
#Data
numSizes = 5
numPatterns = 13
#maxNumRolls = 94
#width = {1:15, 2:20, 3:24, 4:30, 5:40}
demand = {1:19, 2:22, 3:32, 4:14,5:7}
a = { (1,1):4, (1,2):0, (1,3):0 ,(1,4):0, (1,5):0 ,
(2,1):2, (2,2):1, (2,3):0, (2,4):0, (2,5):0,
(3,1):2, (3,2):0, (3,3):1, (3,4):0, (3,5):0,
(4,1):2, (4,2):0, (4,3):0, (4,4):1, (4,5):0,
(5,1):1, (5,2):2, (5,3):0, (5,4):0, (5,5):0,
(6,1):1, (6,2):1, (6,3):1, (6,4):0, (6,5):0,
(7,1):1, (7,2):0, (7,3):0, (7,4):0, (7,5):1,
(8,1):0, (8,2):3, (8,3):0, (8,4):0, (8,5):0,
(9,1):0, (9,2):1, (9,3):0, (9,4):1, (9,5):0,
(10,1):0, (10,2):1, (10,3):0, (10,4):0, (10,5):1,
(11,1):0, (11,2):0, (11,3):2, (11,4):0, (11,5):0,                   
(12,1):0, (12,2):0, (12,3):1, (12,4):1, (12,5):0, 
(13,1):0, (13,2):0, (13,3):0, (13,4):2, (13,5):0}

In [2]:
import pyomo.environ as pe
import pyomo.opt as po

In [3]:
model = pe.ConcreteModel()

In [4]:
model.P = pe.RangeSet(1, numPatterns)
model.S = pe.RangeSet(1, numSizes)

In [5]:
model.z = pe.Var(model.P, domain = pe.NonNegativeIntegers)

In [6]:
objExpr = sum(model.z[p] for p in model.P)
model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [7]:
## Constraints
model.cnstrS = pe.ConstraintList()
for s in model.S:
    expression = sum(model.z[p] * a[p,s]  for p in model.P) >= demand[s]
    model.cnstrS.add(expression)

In [8]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-12-10
Read LP format model from file C:\Users\rpo330\AppData\Local\Temp\tmpvz5353rr.pyomo.lp
Reading time = 0.01 seconds
x14: 6 rows, 14 columns, 24 nonzeros
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 14 columns and 24 nonzeros
Model fingerprint: 0xc4cbf59e
Variable types: 1 continuous, 13 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 53.0000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 5 rows, 13 columns, 23 nonzeros
Variable types: 0 continuous, 13 integer (0 binary)

Root relaxation: objective 3.850000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    | 

In [9]:
print(result.solver.status)
print(result.solver.termination_condition)

ok
optimal


In [10]:
print("Objective value = "+ str(pe.value(model.obj)))

print("Rolls:")
for p in model.P:
    if(pe.value(model.z[p]) > 0):
        print(str(model.z[p]) + " = " + str(pe.value(model.z[p])))

Objective value = 39.0
Rolls:
z[1] = 1.0
z[6] = 15.0
z[10] = 7.0
z[11] = 9.0
z[13] = 7.0
